# Importing Libraries

In [228]:
# importing of library
import pandas as pd
import numpy as np 

import re
from email import message_from_string
from email.utils import parseaddr
from typing import List, Optional, Dict, Any
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
from IPython.display import display

In [229]:
# Reading the data
data_chunk = pd.read_csv('emails.csv', chunksize=10000)  # read 10k rows at a time
dataset = pd.concat([chunk for chunk in data_chunk]) # concatenate all chunks into a single DataFrame

# Display the dataset
display(dataset.head())

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


# Data Exploration

Exploring the dataset helps us better understand its structure and characteristics.

This dataset contains approximately **517,401 emails** from about 150 Enron employees, mostly senior management. It was originally collected and prepared by the CALO Project and later made public by the Federal Energy Regulatory Commission during its investigation. Some messages were deleted or redacted, and invalid email addresses were standardized. This dataset does **not include attachments**.  

For this section, we will follow these steps:

1. Access a sample email from the dataset (first, middle, and last)  
2. Generate descriptive statistics  
3. Handle missing/null values  
4. Check for duplicate rows  
5. Check for empty emails  
6. Check for emails containing non-ASCII characters  


### Accessing Sample Emails from the Dataset (First, Middle, and Last)

The dataset contains 517,401 rows (indexed 0 to 517,400).  
We will examine the first, middle, and last emails to inspect their structure and determine the cleaning steps required.

In [230]:
# Accessing the content of the first email at index 0
print(dataset["message"][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [231]:
# Accessing the content of the middle email at index 258700
print(dataset["message"][258700])

Message-ID: <6456001.1075840867645.JavaMail.evans@thyme>
Date: Wed, 11 Jul 2001 13:22:18 -0700 (PDT)
From: jean.mrha@enron.com
To: louise.kitchen@enron.com
Subject: Hot List Update
Cc: tammie.schoppe@enron.com, melissa.jones@enron.com
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
Bcc: tammie.schoppe@enron.com, melissa.jones@enron.com
X-From: Mrha, Jean </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JMRHA>
X-To: Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Lkitchen>
X-cc: Schoppe, Tammie </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Tstaggs>, Jones, Melissa </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Mjones1>
X-bcc: 
X-Folder: \ExMerge - Kitchen, Louise\'Americas\Mrha
X-Origin: KITCHEN-L
X-FileName: louise kitchen 2-7-02.pst

Please see attached.

 


In [232]:
# Accessing the content of the last email at index 517400
print(dataset["message"][517400])

Message-ID: <28618979.1075842030037.JavaMail.evans@thyme>
Date: Mon, 26 Nov 2001 10:48:43 -0800 (PST)
From: john.zufferli@enron.com
To: livia_zufferli@monitor.com
Subject: RE: ali's essays
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JZUFFER>
X-To: 'Livia_Zufferli@Monitor.com@ENRON'
X-cc: 
X-bcc: 
X-Folder: \ExMerge - Zufferli, John\Sent Items
X-Origin: ZUFFERLI-J
X-FileName: john zufferli 6-26-02.PST

i think the YMCA has a class that is for people recovering from heart-attacks
i remeber something about that

 -----Original Message-----
From: 	Livia_Zufferli@Monitor.com@ENRON  
Sent:	Monday, November 26, 2001 11:44 AM
To:	Zufferli, John
Subject:	RE: ali's essays


i don't know about the heart classes.  i'll look into it, but her dr (ravi)
isn't offering up any suggestions or anything.  she saw him before the
surgery in august, and he said things were okay.  i really don't think he's

From inspecting the first, middle, and last emails, we can see the general structure and content of the dataset.  

Key observations include:
- Emails contain extensive headers and metadata, which are not needed for text analysis.
- Some emails may have empty subjects or body content.
- There is inconsistent formatting, including line breaks, tabs, and spaces, which will need cleaning.
- All emails appear to use standard ASCII encoding, but we will still check for encoding issues.

These insights help us identify potential issues and guide the next steps in cleaning and parsing the dataset. Before proceeding, we will continue with data exploration to gain a better understanding of the dataset.

### Descriptive Statistics 

In [233]:
# Make a copy to prevent mutation
# dataset_ds = dataset.copy()

# Shape of dataset
print(dataset.shape)

(517401, 2)


In [234]:
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB
None


### Handling Missing Values

In [235]:
# Check for missing values in the dataframe
print(dataset.isna().sum().sort_values())

file       0
message    0
dtype: int64


### Check for Duplicate Rows

In [236]:
# Removing duplicate rows
dataset = dataset.drop_duplicates(subset=["message"]).reset_index(drop=True)

# Shape of dataset after removing duplicates
print(f"Shape after removing duplicates: {dataset.shape}")

Shape after removing duplicates: (517401, 2)


Based on the dataset summary from `info()`, all 517,401 emails have non-null values, so there are no missing entries.  However, this does not guarantee that all emails contain meaningful content, as some messages could be completely empty.  Therefore, we perform a check to identify any emails with empty message bodies.

### Check for empty emails

In [237]:
# Check for completely empty emails without removing spaces for parsing
empty_rows = dataset[dataset['message'] == ""]
print(f"Number of completely empty emails: {empty_rows.shape[0]}")

Number of completely empty emails: 0


### Check to see if there is any emails in non-ASCII Characters

In [238]:
# Function to check if a text contains any non-ASCII characters
def has_non_ascii(text):
    return any(ord(c) > 127 for c in str(text))

# Apply to the 'message' column
non_ascii_rows = dataset[dataset['message'].apply(has_non_ascii)]
print(f"Number of emails with non-ASCII characters: {non_ascii_rows.shape[0]}")


Number of emails with non-ASCII characters: 90


In [239]:
print(non_ascii_rows["message"].iloc[0])

Message-ID: <13235995.1075840348885.JavaMail.evans@thyme>
Date: Wed, 23 Jan 2002 19:39:18 -0800 (PST)
From: bgibson50606@houston.rr.com
To: sumpter.teresa@enron.com, stone.pam@enron.com, smithey.linda@enron.com, 
	sinitiere.mary�nelle@enron.com, shrode.cindy@enron.com, 
	short.suzanne@enron.com, scardello.jackie@enron.com, 
	roberts.gina@enron.com, richmond.trisch@enron.com, 
	reese.lisa@enron.com, ramsey.nancy@enron.com, 
	purser.brenda@enron.com, price.lisa@enron.com, price.jane@enron.com, 
	poullard.marie@enron.com, podraza.judy@enron.com, 
	petrak.janet@enron.com, pearce.becky@enron.com, 
	pauley.sharon@enron.com, passero.colleen@enron.com, 
	organ.kathryn@enron.com, munn.mary@enron.com, morgan.tia@enron.com, 
	montalvo.meg@enron.com, molohon.nora@enron.com, mikel.val@enron.com, 
	mcmullen.katie@enron.com, mclaughlin.patricia@enron.com, 
	mcguinness.lori@enron.com, mcdaniel.teri@enron.com, 
	mccracken.claudia@enron.com, matthews.patricia�@enron.com, 
	martin.joy@enron.com, marshall


Based on the data exploration, we observed that the dataset contains no null values, no duplicate rows, and no completely empty emails. However, we did identify a small number of emails containing non-ASCII characters. Since we are building a phishing email detection system, we have decided **not to remove these emails** and will handle them appropriately during system development. We believe this is beneficial, as these emails may help detect unusual or potentially suspicious messages while also verifying legitimate ones.  

Next, we proceed to clean the dataset to prepare the emails for parsing and analysis.

# Data Cleaning

In this section, we will clean the dataset using several methods:

1. Email Parsing  
2. Text Cleaning
3. Post-Parsing Data Checks

**Email parsing** involves extracting the meaningful content from each email, such as the body text, while removing unnecessary components like headers, metadata, or special formatting.

This step is essential to prepare the emails for further cleaning, analysis, or natural language processing tasks.


### Email Parsing
Email parsing is essential to extract structured information from raw emails.  
We will split this process into three main sections:

1. **Header extraction:** Important fields like `Message-ID`, `Date`, `From`, `To`, `Subject`, `X-From`, and `X-To` will be extracted from the email headers.   
2. **Message body extraction:** The main content of the email will be isolated for further analysis, including text cleaning and phishing detection. 
3. **URL extraction:** Links are crucial for identifying suspicious or malicious content.


In [240]:
# transform the email into correct format
message = dataset.loc[0]['message']
e = message_from_string(message)

e.items()

[('Message-ID', '<18782981.1075855378110.JavaMail.evans@thyme>'),
 ('Date', 'Mon, 14 May 2001 16:39:00 -0700 (PDT)'),
 ('From', 'phillip.allen@enron.com'),
 ('To', 'tim.belden@enron.com'),
 ('Subject', ''),
 ('Mime-Version', '1.0'),
 ('Content-Type', 'text/plain; charset=us-ascii'),
 ('Content-Transfer-Encoding', '7bit'),
 ('X-From', 'Phillip K Allen'),
 ('X-To', 'Tim Belden <Tim Belden/Enron@EnronXGate>'),
 ('X-cc', ''),
 ('X-bcc', ''),
 ('X-Folder', "\\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail"),
 ('X-Origin', 'Allen-P'),
 ('X-FileName', 'pallen (Non-Privileged).pst')]

After inspecting the content of a sample email, we observed that each email contains useful information in its headers, such as `Message-ID`, `Date`, `From`, `To`, `Subject`, and employee metadata (`X-From`, `X-To`).  

To facilitate further analysis and rule-based phishing detection, we decided to extract these fields from all emails and store them in a structured DataFrame.

In [241]:
def parse_email(raw_msg: Any, fields: Optional[List[str]] = None) -> Dict[str, Optional[str]]:
    """
    Parse a raw email string and extract specified header fields.

    Parameters
    ----------
    raw_msg : str
        The raw email content as a string.
    fields : list[str], optional
        List of email fields to extract. Default includes common fields.

    Returns
    -------
    dict
        Dictionary mapping field names to extracted values (or None if missing).
    """
    if fields is None:
        fields = ["Message-ID", "Date", "From", "To", "Subject", "X-From", "X-To"]

    try:
        email_obj = message_from_string(raw_msg)
        return {field.lower().replace("-", "_"): email_obj.get(field) 
                for field in fields}
    except Exception:
        # Return None for all fields if parsing fails
        return {field.lower().replace("-", "_"): None for field in fields}


def build_email_dataframe(df: pd.DataFrame, message_col: str = "message", fields: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Extract structured fields from raw email messages in a DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing a column of raw email text.
    message_col : str
        Column name containing raw email strings.
    fields : list[str], optional
        Fields to extract from each email.

    Returns
    -------
    pd.DataFrame
        DataFrame with extracted email fields.
    """
    # tqdm progress bar for large datasets
    parsed = df[message_col].apply(lambda msg: parse_email(msg, fields))
    parsed = list(tqdm(parsed, total=len(df), desc="Parsing emails"))
    return pd.DataFrame(parsed)

# Extract fields from the raw message column using the build_email_dataframe function
extracted_df = build_email_dataframe(dataset, message_col="message") # Did not specify fields, so default fields will be extracted

Parsing emails: 100%|██████████| 517401/517401 [00:00<00:00, 2081155.21it/s]


#### Message Body Extraction

In [242]:
def body(messages):
    column = []
    for message in messages:
        e = message_from_string(message)
        column.append(e.get_payload())
    return column

extracted_df['body'] = body(dataset['message'])

### Validation of Body and Header Extraction  

After extracting the body of each email, it is important to validate the results. Due to inconsistencies and formatting issues within the Enron dataset, some emails may not parse correctly. This can lead to:  

1. **Incomplete or incorrect body extraction**  
   - In certain cases, parts of the email headers may still remain inside the `body` field instead of being fully separated.  
   - This requires manual or programmatic checks to confirm that the `body` column truly contains only the message content.  

2. **Null or missing values in headers**  
   - Some header fields such as `to`, `from`, or `subject` may appear as null after parsing.  
   - These values may still exist within the raw email text but were not properly extracted during parsing.  

To address this, we will:  
- Inspect a sample of emails to verify that the `body` field contains the actual message rather than residual headers.  
- Cross-check the raw `message` text for cases where header fields (e.g., `to`) are null, and attempt to recover these values if possible.  

This step ensures that the dataset is **accurately structured** before proceeding to further cleaning and analysis.  

In [243]:
# # Function to clean body after last X-FileName and recover X-From / X-To if present
# def clean_body_after_xfilename(raw_body):
#     """
#     Process order:
#       1) Normalize newlines and split into lines.
#       2) Scan ALL lines first to recover X-From / X-To (case-insensitive).
#       3) Find the last 'X-FileName:' anchor and take the body ONLY after it.
#       4) From that slice, drop any residual X-From / X-To lines.
#       5) Join and trim → return (body_clean, recovered_dict).
#     """
#     if raw_body is None:
#         return "", {}

#     # 1) Normalize newlines and split into lines
#     lines = str(raw_body).replace("\r\n", "\n").replace("\r", "\n").split("\n")

#     # 2) Recover X-From / X-To from ANYWHERE in the text (headers/body)
#     recovered = {}
#     xhdr = re.compile(r'^(x-from|x-to)\s*:\s*(.*)$', re.IGNORECASE)
#     for ln in lines:
#         m = xhdr.match(ln.strip())
#         if m:
#             key = m.group(1).lower().replace("-", "_")   # -> "x_from" / "x_to"
#             recovered[key] = m.group(2).strip()

#     # 3) Find last 'X-FileName:' anchor; body starts AFTER this line
#     anchor_idx = -1
#     for i, ln in enumerate(lines):
#         if re.match(r'(?i)^x-filename\s*:', ln.strip()):
#             anchor_idx = i
#     body_slice = lines[anchor_idx + 1:] if anchor_idx >= 0 else lines

#     # 4) From the body slice, drop any X-From / X-To lines (we’ve already captured them)
#     kept = []
#     for ln in body_slice:
#         if xhdr.match(ln.strip()):
#             continue
#         kept.append(ln)

#     # 5) Join → tidy a bit (optional) → return
#     body = "\n".join(kept).strip()
#     return body, recovered

# # ensure columns exist
# for col in ("x_from", "x_to"):
#     if col not in extracted_df.columns:
#         extracted_df[col] = np.nan

# clean_bodies = []
# filled_from = filled_to = 0

# for i, msg in enumerate(extracted_df["body"]):  
#     body_clean, rec = clean_body_after_xfilename(msg)
#     clean_bodies.append(body_clean)

#     if "x_from" in rec and (pd.isna(extracted_df.at[i, "x_from"]) or str(extracted_df.at[i, "x_from"]).strip() == ""):
#         extracted_df.at[i, "x_from"] = rec["x_from"]; filled_from += 1
#     if "x_to" in rec and (pd.isna(extracted_df.at[i, "x_to"]) or str(extracted_df.at[i, "x_to"]).strip() == ""):
#         extracted_df.at[i, "x_to"] = rec["x_to"];     filled_to += 1

# extracted_df["body"] = clean_bodies
# print(f"Filled x_from in {filled_from} rows; x_to in {filled_to} rows.")


In [ ]:
# --- core: parse one body string ---
def extract_body_and_x(body_text: str):
    """
    Returns (clean_body, x_from, x_to).

    Steps (simple):
    1) Normalize newlines and split into lines.
    2) Remember the last seen 'X-From:' and 'X-To:' (anywhere).
    3) Body starts AFTER the last 'X-FileName:' line.
    4) Remove any X-From/X-To lines from that body slice.
    """
    if body_text is None:
        return "", None, None

    # 1) normalize & split
    lines = str(body_text).replace("\r\n", "\n").replace("\r", "\n").split("\n")

    # 2) capture X-From / X-To (keep last value if multiple)
    x_from = x_to = None
    xhdr = re.compile(r'^(x-from|x-to)\s*:\s*(.*)$', re.IGNORECASE)
    for ln in lines:
        m = xhdr.match(ln.strip())
        if m:
            key, val = m.group(1).lower(), m.group(2).strip()
            if key == "x-from": x_from = val
            else:               x_to   = val

    # 3) find last 'X-FileName:' anchor; body starts after it
    anchor = -1
    for i, ln in enumerate(lines):
        if re.match(r'(?i)^x-filename\s*:', ln.strip()):
            anchor = i
    body_slice = lines[anchor + 1:] if anchor >= 0 else lines

    # 4) drop any X-From/X-To lines from body
    kept = [ln for ln in body_slice if not xhdr.match(ln.strip())]
    body_clean = "\n".join(kept).strip()

    return body_clean, x_from, x_to


# --- apply to a dataframe (reusable wrapper) ---
def apply_extract_body_and_x(df: pd.DataFrame,
                             body_col: str = "body",
                             x_from_col: str = "x_from",
                             x_to_col: str = "x_to") -> dict:
    """
    Cleans df[body_col], fills missing x_from/x_to, and returns counts.
    - Updates df[body_col] in place with the cleaned body.
    - Creates x_from/x_to columns if they don't exist.
    - Fills only where current value is empty/NaN.
    """
    # ensure target columns exist
    for col in (x_from_col, x_to_col):
        if col not in df.columns:
            df[col] = np.nan

    # run once, expand into columns
    res = df[body_col].apply(extract_body_and_x).apply(pd.Series)
    res.columns = ["__body_tmp__", "__x_from_tmp__", "__x_to_tmp__"]

    # helper: is empty cell?
    is_empty = lambda s: s.isna() | (s.astype(str).str.strip() == "")

    # fill counts
    mask_from = is_empty(df[x_from_col]) & res["__x_from_tmp__"].notna()
    mask_to   = is_empty(df[x_to_col])   & res["__x_to_tmp__"].notna()

    # update df
    df.loc[:, body_col] = res["__body_tmp__"]
    df.loc[mask_from, x_from_col] = res.loc[mask_from, "__x_from_tmp__"]
    df.loc[mask_to,   x_to_col]   = res.loc[mask_to,   "__x_to_tmp__"]

    # cleanup temp cols (not added to df, they are in res only)
    return {
        "filled_x_from": int(mask_from.sum()),
        "filled_x_to":   int(mask_to.sum()),
        "rows":          int(len(df))
    }

stats = apply_extract_body_and_x(extracted_df, body_col="body", x_from_col="x_from", x_to_col="x_to")
print(stats)  # {'filled_x_from': 29, 'filled_x_to': 17, 'rows': 517401}  (example numbers)

{'filled_x_from': 29, 'filled_x_to': 29, 'rows': 517401}


In [245]:
extracted_df['body'][18162]

'Fundamental Overview Presentation\nFriday, September 21st\n1 - 3pm\nroom 32c2'

#### URL Extraction

Next, we extract all URLs contained in the email bodies.  

URLs are important for phishing detection because suspicious or malicious links are often key indicators of phishing attempts.  

By isolating the URLs, we can analyze them separately and apply rules to identify potentially harmful links.


In [246]:
# Function to extract URLs from dataset['message'] directly
def extract_urls_from_message(raw_msg):
    if not isinstance(raw_msg, str):
        return None
    url_pattern = r'(https?://[^\s]+)'
    urls = re.findall(url_pattern, raw_msg)
    return urls if urls else None

# apply directly on the raw message column
extracted_df['urls'] = dataset['message'].apply(extract_urls_from_message)
extracted_df['num_urls'] = extracted_df['urls'].apply(lambda x: len(x) if x is not None else 0)

In [247]:
# Display the first few rows to verify extraction
display(extracted_df.head())

,message_id,date,from,to,subject,x_from,x_to,body,urls,num_urls
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast,None,0
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...,None,0
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!,None,0
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,Phillip K Allen,Randall L Gay,"Randy,\n\n Can you send me a schedule of the s...",None,0
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.,None,0


Based on the extracted dataset, all key headers, the email body and URLs have been successfully captured and standardized. The `body` field is readable and normalized, while `message_id` preserves its original format. Although this sample shows no URLs, the dataset is structured to capture them if present in other emails. 

### Text Cleaning

In this step, we clean the relevant text fields in the dataset to prepare for analysis and phishing detection.  

The cleaning process includes:
1. **Removing content inside angle brackets (`<...>`)** for all columns except `message_id`.  
   - This helps standardize email addresses and header fields.  
2. **Normalizing whitespace**  
   - Multiple spaces, tabs, and newlines are replaced with a single space.  
   - Leading and trailing spaces are removed.   
3. **Reordering and renaming columns**  
   - Adjust column order and names to match the workflow for phishing detection analysis.  
   - This makes the dataset more organized and easier to work with for subsequent steps.

This ensures all text fields are **clean, consistent, and ready** for further processing, while preserving important information for phishing detection, including URLs, attachments, and non-ASCII characters.


In [248]:
# Make a copy to prevent mutation
data_ds = extracted_df.copy()

def clean_text(x, keep_tags=False):
    """
    - Collapse whitespace
    - Remove <...> entirely unless keep_tags=True
    - Remove [] but keep the content inside
    - Remove quotes ' and "
    """
    if x is None:
        return None # No change

    text = str(x)

    if not keep_tags:
        text = re.sub(r'<[^>]*>', '', text)      # remove <...>

    text = re.sub(r'[\[\]\'"]+', '', text)       # drop [, ], ', "

    return re.sub(r'\s+', ' ', text).strip()     # collapse ws + strip

# now apply depending on the column 
for col in ['message_id', 'date', 'from', 'x_from', 'to', 'x_to', 'subject', 'body', 'urls', 'num_urls']:
    if col == 'message_id':
        data_ds[col] = data_ds[col].apply(lambda x: clean_text(x, keep_tags=True))
    else:
        data_ds[col] = data_ds[col].apply(clean_text)

In [249]:
# # Optional: Remove angle brackets from message_id
# data_ds['message_id'] = data_ds['message_id'].str.replace(r'[<>]', '', regex=True)

In [250]:
# Rearranging columns for better readability
cols = ['message_id', 'date', 'from', 'x_from', 'to', 'x_to', 'subject', 'body', 'urls', 'num_urls']
data_ds = data_ds[cols]

# Changing column names for better readability
data_ds = data_ds.rename(columns={
    'x_from': 'sender',
    'x_to': 'recipient'
})

In [251]:
# display the cleaned dataframe
display(data_ds.head())

,message_id,date,from,sender,to,recipient,subject,body,urls,num_urls
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,Phillip K Allen,tim.belden@enron.com,Tim Belden,,Here is our forecast,None,0
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,Phillip K Allen,john.lavorato@enron.com,John J Lavorato,Re:,Traveling to have a business meeting takes the...,None,0
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,Phillip K Allen,leah.arsdall@enron.com,Leah Van Arsdall,Re: test,test successful. way to go!!!,None,0
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,Phillip K Allen,randall.gay@enron.com,Randall L Gay,,"Randy, Can you send me a schedule of the salar...",None,0
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,Phillip K Allen,greg.piper@enron.com,Greg Piper,Re: Hello,Lets shoot for Tuesday at 11:45.,None,0


### Post-Parsing Data Validation

After parsing and splitting the emails into separate columns, it is important to verify the integrity of the new dataset.  

We will:

1. **Check for null values** – Some fields such as `subject` or `body` may be empty even if the original message was not null.  
2. **Check for duplicate rows** – Parsing may create redundant entries that should be removed.  
3. **Inspect dataset summary** – Using `data.info()` to review column names, data types, and non-null counts.  

These steps ensure that the parsed dataset is **clean, consistent, and ready** for further analysis and phishing detection.

#### Descriptive Statistics

In [252]:
# Check the info of cleaned dataframe
data_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   message_id  517401 non-null  object
 1   date        517401 non-null  object
 2   from        517401 non-null  object
 3   sender      517401 non-null  object
 4   to          495554 non-null  object
 5   recipient   517401 non-null  object
 6   subject     517401 non-null  object
 7   body        517401 non-null  object
 8   urls        67121 non-null   object
 9   num_urls    517401 non-null  object
dtypes: object(10)
memory usage: 39.5+ MB


##### Handling of Null Values

In [253]:
# Check for missing values in the dataframe
print(data_ds.isna().sum().sort_values())

message_id         0
date               0
from               0
sender             0
recipient          0
subject            0
body               0
num_urls           0
to             21847
urls          450280
dtype: int64


##### Checking for duplicates

In [254]:
# Removing duplicate rows
data_ds = data_ds.drop_duplicates().reset_index(drop=True)

# Shape of dataset after removing duplicates
print(f"Shape after removing duplicates: {data_ds.shape}")

Shape after removing duplicates: (517401, 10)


In [255]:
# # Make a copy to prevent mutation
# data_ds = extracted_df.copy()

# def clean_text(x, keep_tags=False):
#     """
#     - Collapse whitespace
#     - Remove <...> entirely unless keep_tags=True
#     - Remove [] but keep the content inside
#     - Remove quotes ' and "
#     """
#     if x is None:
#         return None # No change

#     text = str(x)

#     if not keep_tags:
#         text = re.sub(r'<[^>]*>', '', text)      # remove <...>

#     text = re.sub(r'[\[\]\'"]+', '', text)       # drop [, ], ', "

#     return re.sub(r'\s+', ' ', text).strip()     # collapse ws + strip

# clean_text('Baughman Jr., Don </O=ENRON/OU=NA/CN=RECIPIENTS/CN=DBAUGHM>')